<a href="https://colab.research.google.com/github/ehddnr301/dacon_cv2/blob/master/dacon_computervision2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Mon Feb 15 01:30:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os

os.listdir('/content/drive/MyDrive/dacon_computer_vision/')

['data',
 'Untitled1.ipynb',
 'model.h5',
 'eff_model.pt',
 'eff_model1.pt',
 'dacon-vision2.ipynb',
 'Untitled0.ipynb',
 'checkpoint',
 'dacon_cv.zip',
 'eff_model_cv2.pt',
 'Untitled3.ipynb',
 'Untitled2.ipynb',
 'dacon_cv2.ipynb',
 'eff_model_0214.pt',
 'dacon_computervision2.ipynb']

In [4]:
# !unzip -n '/content/drive/MyDrive/dacon_computer_vision/dacon_cv.zip' -d '/content/drive/MyDrive/dacon_computer_vision/data/'

In [ ]:
# colab 에서 자꾸 unzip한 이미지중에 일부가 없어져서 -n modifier를 주고 계속 실행하려함.
!unzip -n '/content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist_2nd.zip' -d '/content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist'

Archive:  /content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist_2nd.zip


In [ ]:
!pip install torchinfo

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
import os
from typing import Tuple, Sequence, Callable
import csv
import cv2
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torchinfo import summary

import albumentations


from torchvision import transforms


In [ ]:
# random seed
random_seed = 777
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)


IMAGE_WIDTH=256
IMAGE_HEIGHT=256
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
EPOCHS=40

PATH_TRAIN_DATASET='/content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist/'
PATH_TEST_DATASET='/content/drive/MyDrive/dacon_computer_vision/data/test_dirty_mnist/'
PATH_TRAIN_ANS_CSV='/content/drive/MyDrive/dacon_computer_vision/data/dirty_mnist_2nd_answer.csv'

In [ ]:
df = pd.read_csv(PATH_TRAIN_ANS_CSV)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.05, random_state=random_seed)

In [ ]:
train.head()

In [ ]:
PATH_TRAIN_CSV='/content/drive/MyDrive/dacon_computer_vision/data/mnist_train.csv'
PATH_VALID_CSV='/content/drive/MyDrive/dacon_computer_vision/data/mnist_valid.csv'


In [ ]:
train.to_csv(PATH_TRAIN_CSV, index=False)

In [ ]:
valid.to_csv(PATH_VALID_CSV, index=False)

In [ ]:
import random
import matplotlib.pyplot as plt


plt.figure(figsize=(12,6))
mnist_sample = random.sample(os.listdir(PATH_TRAIN_DATASET), 10)

for i, image in enumerate(mnist_sample):
  path = os.path.join(PATH_TRAIN_DATASET, image)
  img = Image.open(path)
  plt.subplot(2,5,i+1)
  plt.imshow(img)
  plt.xlabel('mnist')
  plt.xticks([])
  plt.yticks([])

plt.tight_layout()


In [ ]:
class MnistDataset(Dataset):
    def __init__(
        self,
        dir: os.PathLike,
        image_ids: os.PathLike,
        transforms: Sequence[Callable]
    ) -> None:
        self.dir = dir
        self.transforms = transforms

        self.labels = {}
        with open(image_ids, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                self.labels[int(row[0])] = list(map(int, row[1:]))

        self.image_ids = list(self.labels.keys())

    def __len__(self) -> int:
        return len(self.image_ids)

    def __getitem__(self, index: int) -> Tuple[Tensor]:
        image_id = self.image_ids[index]
        image = Image.open(
            os.path.join(
                self.dir, f'{str(image_id).zfill(5)}.png')).convert('RGB')
        target = np.array(self.labels.get(image_id)).astype(np.float32)

        if self.transforms is not None:
            augmented = self.transform(image=image) 
            image = augmented['image']
            
        return image, target

In [ ]:
transforms_train = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.ShiftScaleRotate(p=0.5,rotate_limit=180),
    albumentations.OneOf([
                          albumentations.RandomBrightness(limit=0.1),
                          albumentations.RandomContrast(),
                          albumentations.Blur()
    ])
    albumentations.CenterCrop(240,240,p=0.3),
    albumentations.pytorch.transforms.ToTensorV2(),
    albumentations.augmentations.transforms.Normalize()
])

transforms_valid = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.pytorch.transforms.ToTensorV2(),
    albumentations.augmentations.transforms.Normalize()
])

transforms_test = albumentations.Compose([
    albumentations.pytorch.transforms.ToTensorV2(),
    albumentations.augmentations.transforms.Normalize()
])


# transforms_train = transforms.Compose([
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomVerticalFlip(p=0.5),
#     transforms.RandomRotation(degrees=(180, 180)),
#     transforms.RandomAffine(5),
#     transforms.RandomGrayscale(p=0.5),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         [0.485, 0.456, 0.406],
#         [0.229, 0.224, 0.225]
#     )
# ])

# transforms_valid = transforms.Compose([
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomVerticalFlip(p=0.5),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         [0.485, 0.456, 0.406],
#         [0.229, 0.224, 0.225]
#     )
# ])

# transforms_test = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(
#         [0.485, 0.456, 0.406],
#         [0.229, 0.224, 0.225]
#     )
# ])

In [ ]:
trainset = MnistDataset(PATH_TRAIN_DATASET, PATH_TRAIN_CSV, transforms_train)
valset = MnistDataset(PATH_TRAIN_DATASET, PATH_VALID_CSV, transforms_valid)
testset = MnistDataset(PATH_TEST_DATASET, '/content/drive/MyDrive/dacon_computer_vision/data/sample_submission.csv', transforms_test)

train_loader = DataLoader(trainset, batch_size=32)
val_loader = DataLoader(valset, batch_size=32)
test_loader = DataLoader(testset, batch_size=16)

In [ ]:
# print(valset.labels)

In [ ]:
class MnistModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.effnet = EfficientNet.from_pretrained('efficientnet-b5')
        self.classifier = nn.Linear(1000, 26)

    def forward(self, x):
        x = self.effnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MnistModel().to(device)
print(summary(model, input_size=(1, 3, 256, 256), verbose=0))

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                  mode='min',
                                                  factor=0.5,
                                                  patience=2,)



for epoch in range(EPOCHS):
    

    for i, (images, targets) in enumerate(train_loader):

        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        model.train()

        with torch.set_grad_enabled(True):
            outputs = model(images)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

        if i % 50 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch+1}: {loss.item():.5f}, {acc.item():.5f}')


    valid_acc_list = []
    with tqdm(val_loader,
            total=val_loader.__len__(),
            unit="batch") as valid_bar:
        for i, (images, targets) in enumerate(valid_bar):
            valid_bar.set_description(f"Valid Epoch {epoch + 1}")
            optimizer.zero_grad()

            images = images.to(device)
            targets = targets.to(device)

            # 모델의 dropoupt, batchnormalization를 eval모드로 설정
            # .forward()에서 중간 노드의 gradient를 계산

            model.eval()

            with torch.no_grad():
                # validation loss만을 계산
                probs  = model(images)
                valid_loss = criterion(probs, targets)


                # train accuracy 계산
                probs  = probs.cpu().detach().numpy()
                targets = targets.cpu().detach().numpy()
                preds = probs > 0.5
                batch_acc = (targets == preds).mean()
                valid_acc_list.append(batch_acc)

            valid_acc = np.mean(valid_acc_list)
            valid_bar.set_postfix(valid_loss = valid_loss.item(),
                                  valid_acc = valid_acc)

    scheduler.step(valid_loss)


    if (epoch + 1) % 5 == 0:
        torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, f'/content/drive/MyDrive/dacon_computer_vision/checkpoint/model_0214_{epoch + 1}.pt')
    
    print('------------------')

In [ ]:
torch.save(model, '/content/drive/MyDrive/dacon_computer_vision/eff_model_0215.pt')

In [ ]:
# 중간에 학습이 종료되어서 6번째 epoch을 불러와서 predict에 사용

# PATH = '/content/drive/MyDrive/dacon_computer_vision/checkpoint/model_0214_20.pt'
# model = MnistModel().to(device)

# checkpoint = torch.load(PATH)
# model.load_state_dict(checkpoint['model_state_dict'])


In [ ]:
!unzip -n '/content/drive/MyDrive/dacon_computer_vision/data/test_dirty_mnist_2nd.zip' -d '/content/drive/MyDrive/dacon_computer_vision/data/test_dirty_mnist'

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/dacon_computer_vision/data/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('/content/drive/MyDrive/dacon_computer_vision/data/submit0215.csv', index=False)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=1e-3)
criterion = nn.MultiLabelSoftMarginLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                  mode='min',
                                                  factor=0.5,
                                                  patience=2,)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])



for epoch in range(10):
    

    for i, (images, targets) in enumerate(val_loader):

        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        model.train()

        with torch.set_grad_enabled(True):
            outputs = model(images)
            loss = criterion(outputs, targets)

            loss.backward()
            optimizer.step()

        if i % 50 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            print(f'{epoch+1}: {loss.item():.5f}, {acc.item():.5f}')


    if (epoch + 1) % 5 == 0:
        torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                }, f'/content/drive/MyDrive/dacon_computer_vision/checkpoint/model_0215_{epoch + 1}.pt')
    
    print('------------------')

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/dacon_computer_vision/data/sample_submission.csv')

model.eval()
batch_size = test_loader.batch_size
batch_index = 0
for i, (images, targets) in enumerate(test_loader):
    images = images.to(device)
    targets = targets.to(device)
    outputs = model(images)
    outputs = outputs > 0.5
    batch_index = i * batch_size
    submit.iloc[batch_index:batch_index+batch_size, 1:] = \
        outputs.long().squeeze(0).detach().cpu().numpy()
    
submit.to_csv('/content/drive/MyDrive/dacon_computer_vision/data/submit0215_after_valid.csv', index=False)